In [3]:
! pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 49.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f8efb68edb97161555d26abbaad9b122ff28216554a60564367003001be79812
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [4]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()
sc = spark.sparkContext

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [69]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.util import MLUtils

import pandas as pd 


# Load and parse the data file, converting it to a DataFrame.
data = spark.read.csv("weatherAUS.csv", inferSchema= True, header = True) 


In [70]:
data = data.drop("Date","Location","Evaporation","Sunshine","Cloud9am",'Cloud3pm','Temp3pm',"Temp9am","Temp3am")

In [71]:
# GET RID OF NA VALUES 

from pyspark.sql.functions import mean
from pyspark.sql import functions as F

for i in data.columns:
  count = data.filter(data[i]=='NA').count()

numeric_column = ['MinTemp','MaxTemp', 'Rainfall','WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm','Humidity9am','Humidity3pm','Pressure9am','Pressure3pm']

# calculate average
for i in numeric_column:
  average = data.select([mean(i)])

# substitute null values with the average
updatedmint = data.withColumn("MinTemp",F.when(F.col("MinTemp") == 'NA',12.2).otherwise(F.col("MinTemp")))
updatedmaxt = updatedmint.withColumn("MaxTemp",F.when(F.col("MaxTemp") == 'NA',23.2).otherwise(F.col("MaxTemp")))
updatedrainf = updatedmaxt.withColumn("Rainfall",F.when(F.col("Rainfall") == 'NA',2.3).otherwise(F.col("Rainfall")))
updatedwind = updatedrainf.withColumn("WindGustSpeed",F.when(F.col("WindGustSpeed") == 'NA',40.0).otherwise(F.col("WindGustSpeed")))
updatedwinds9 = updatedwind.withColumn("WindSpeed9am",F.when(F.col("WindSpeed9am") == 'NA',14.0).otherwise(F.col("WindSpeed9am")))
updatedwinds3 = updatedwinds9.withColumn("WindSpeed3pm",F.when(F.col("WindSpeed3pm") == 'NA',18.6).otherwise(F.col("WindSpeed3pm")))
updatedhum9 = updatedwinds3.withColumn("Humidity9am",F.when(F.col("Humidity9am") == 'NA',68.8).otherwise(F.col("Humidity9am")))
updatedhum3 = updatedhum9.withColumn("Humidity3pm",F.when(F.col("Humidity3pm") == 'NA',51.5).otherwise(F.col("Humidity3pm")))
updatedpres9 = updatedhum3.withColumn("Pressure9am",F.when(F.col("Pressure9am") == 'NA',1017.7).otherwise(F.col("Pressure9am")))


In [72]:
data.printSchema()

root
 |-- MinTemp: string (nullable = true)
 |-- MaxTemp: string (nullable = true)
 |-- Rainfall: string (nullable = true)
 |-- WindGustDir: string (nullable = true)
 |-- WindGustSpeed: string (nullable = true)
 |-- WindDir9am: string (nullable = true)
 |-- WindDir3pm: string (nullable = true)
 |-- WindSpeed9am: string (nullable = true)
 |-- WindSpeed3pm: string (nullable = true)
 |-- Humidity9am: string (nullable = true)
 |-- Humidity3pm: string (nullable = true)
 |-- Pressure9am: string (nullable = true)
 |-- Pressure3pm: string (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [73]:
data.show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|   13.4|   22.9|     0.6|          W|           44|         W|       WNW|          20|          24|         71|         22|     1007.7|     1007.1|       No|          No|
|    7.4|   25.1|       0|        WNW|           44|       NNW|       WSW|           4|          22|         44|         25|     1010.6|     1007.8|       No|          No|
|   12.9|   25.7|       0|        WSW|           46|         W|       WSW|          19|          26|         38|         30|     1007.6|    

In [74]:
data.distinct().show()

+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|MinTemp|MaxTemp|Rainfall|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+---------+------------+
|    4.7|   12.2|       0|        NNW|           24|       ESE|         N|           2|          13|         99|         75|     1015.5|     1012.7|       No|         Yes|
|    1.6|   17.2|       0|          S|           20|       ESE|         S|           6|           7|         68|         44|     1022.5|     1019.1|       No|          No|
|   10.6|   24.7|       0|         NE|           33|        NA|       ENE|           0|          20|         75|         40|     1017.9|    

In [75]:
# tbh idk whats going on, so im just gonna drop more columns....
data = data.drop("MinTemp","MaxTemp","WindGustDir","WindGustSpeed","WindDir9am","WindDir3pm","WindSpeed9am","WindSpeed3pm","Humidity9am","Humidity3pm","Pressure9am","Pressure3pm")
data.show()

+--------+---------+------------+
|Rainfall|RainToday|RainTomorrow|
+--------+---------+------------+
|     0.6|       No|          No|
|       0|       No|          No|
|       0|       No|          No|
|       0|       No|          No|
|       1|       No|          No|
|     0.2|       No|          No|
|       0|       No|          No|
|       0|       No|          No|
|       0|       No|         Yes|
|     1.4|      Yes|          No|
|       0|       No|         Yes|
|     2.2|      Yes|         Yes|
|    15.6|      Yes|         Yes|
|     3.6|      Yes|          No|
|       0|       No|          NA|
|      NA|       NA|          No|
|       0|       No|         Yes|
|    16.8|      Yes|         Yes|
|    10.6|      Yes|          No|
|       0|       No|          No|
+--------+---------+------------+
only showing top 20 rows



In [76]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCols=["RainTomorrow","RainToday","Rainfall"], 
                             outputCols=["tomorrow_index","today_index","rain_index"]).fit(data).transform(data)

In [77]:
# rain tomorrow index: yes = 1.0, no = 0.0, NA = 2.0

labelIndexer.show()

+--------+---------+------------+--------------+-----------+----------+
|Rainfall|RainToday|RainTomorrow|tomorrow_index|today_index|rain_index|
+--------+---------+------------+--------------+-----------+----------+
|     0.6|       No|          No|           0.0|        0.0|       4.0|
|       0|       No|          No|           0.0|        0.0|       0.0|
|       0|       No|          No|           0.0|        0.0|       0.0|
|       0|       No|          No|           0.0|        0.0|       0.0|
|       1|       No|          No|           0.0|        0.0|       6.0|
|     0.2|       No|          No|           0.0|        0.0|       1.0|
|       0|       No|          No|           0.0|        0.0|       0.0|
|       0|       No|          No|           0.0|        0.0|       0.0|
|       0|       No|         Yes|           1.0|        0.0|       0.0|
|     1.4|      Yes|          No|           0.0|        1.0|       8.0|
|       0|       No|         Yes|           1.0|        0.0|    

In [78]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol="tomorrow_index",
                        outputCol="tomorrow_onehot", dropLast=False)
models = encoder.fit(labelIndexer)
encoded = models.transform(labelIndexer)


In [79]:
encoded.show()

# TO UNDERSTAND: 
# tomorrow_onehot = (3,       (rain? yes=1,no=0,NA=[]),      1.0)

+--------+---------+------------+--------------+-----------+----------+---------------+
|Rainfall|RainToday|RainTomorrow|tomorrow_index|today_index|rain_index|tomorrow_onehot|
+--------+---------+------------+--------------+-----------+----------+---------------+
|     0.6|       No|          No|           0.0|        0.0|       4.0|  (3,[0],[1.0])|
|       0|       No|          No|           0.0|        0.0|       0.0|  (3,[0],[1.0])|
|       0|       No|          No|           0.0|        0.0|       0.0|  (3,[0],[1.0])|
|       0|       No|          No|           0.0|        0.0|       0.0|  (3,[0],[1.0])|
|       1|       No|          No|           0.0|        0.0|       6.0|  (3,[0],[1.0])|
|     0.2|       No|          No|           0.0|        0.0|       1.0|  (3,[0],[1.0])|
|       0|       No|          No|           0.0|        0.0|       0.0|  (3,[0],[1.0])|
|       0|       No|          No|           0.0|        0.0|       0.0|  (3,[0],[1.0])|
|       0|       No|         Yes

In [80]:
import pyspark.sql.functions as f
# Split the data 80/20 train/test, using a seed of 12345
(trainingData, testData) = data.randomSplit([0.8, 0.2],seed = 12345)
print(f"Train set length: {trainingData.count()} records")
print(f"Train set length: {testData.count()} records")

Train set length: 116263 records
Train set length: 29197 records


In [81]:
encoded.describe()

DataFrame[summary: string, Rainfall: string, RainToday: string, RainTomorrow: string, tomorrow_index: string, today_index: string, rain_index: string]

In [18]:
# making sure target variable didnt get messed up....
data.groupBy("RainTomorrow").count().show()
encoded.groupBy("RainTomorrow").count().show()

+------------+------+
|RainTomorrow| count|
+------------+------+
|          NA|  3267|
|          No|110316|
|         Yes| 31877|
+------------+------+

+------------+------+
|RainTomorrow| count|
+------------+------+
|          NA|  3267|
|          No|110316|
|         Yes| 31877|
+------------+------+



In [82]:
data.describe().show()

+-------+------------------+---------+------------+
|summary|          Rainfall|RainToday|RainTomorrow|
+-------+------------------+---------+------------+
|  count|            145460|   145460|      145460|
|   mean|2.3609181499168272|     null|        null|
| stddev| 8.478059737726392|     null|        null|
|    min|                 0|       NA|          NA|
|    max|                NA|      Yes|         Yes|
+-------+------------------+---------+------------+



In [89]:
# LET ME USE A VECTOR ASSEMBLY INSTEADDDD
data.columns

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["tomorrow_index","today_index","rain_index"],
                            outputCol= "features")

In [90]:
assembler

Exception ignored in: <function JavaWrapper.__del__ at 0x7fb550df0200>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


VectorAssembler_cb5cf186d465

In [91]:
output = assembler.transform(labelIndexer)

output.show()

+--------+---------+------------+--------------+-----------+----------+--------------+
|Rainfall|RainToday|RainTomorrow|tomorrow_index|today_index|rain_index|      features|
+--------+---------+------------+--------------+-----------+----------+--------------+
|     0.6|       No|          No|           0.0|        0.0|       4.0| [0.0,0.0,4.0]|
|       0|       No|          No|           0.0|        0.0|       0.0|     (3,[],[])|
|       0|       No|          No|           0.0|        0.0|       0.0|     (3,[],[])|
|       0|       No|          No|           0.0|        0.0|       0.0|     (3,[],[])|
|       1|       No|          No|           0.0|        0.0|       6.0| [0.0,0.0,6.0]|
|     0.2|       No|          No|           0.0|        0.0|       1.0| [0.0,0.0,1.0]|
|       0|       No|          No|           0.0|        0.0|       0.0|     (3,[],[])|
|       0|       No|          No|           0.0|        0.0|       0.0|     (3,[],[])|
|       0|       No|         Yes|          

In [92]:
output.select("features", "tomorrow_index").show()

+--------------+--------------+
|      features|tomorrow_index|
+--------------+--------------+
| [0.0,0.0,4.0]|           0.0|
|     (3,[],[])|           0.0|
|     (3,[],[])|           0.0|
|     (3,[],[])|           0.0|
| [0.0,0.0,6.0]|           0.0|
| [0.0,0.0,1.0]|           0.0|
|     (3,[],[])|           0.0|
|     (3,[],[])|           0.0|
| [1.0,0.0,0.0]|           1.0|
| [0.0,1.0,8.0]|           0.0|
| [1.0,0.0,0.0]|           1.0|
|[1.0,1.0,12.0]|           1.0|
|[1.0,1.0,81.0]|           1.0|
|[0.0,1.0,20.0]|           0.0|
| [2.0,0.0,0.0]|           2.0|
| [0.0,2.0,3.0]|           0.0|
| [1.0,0.0,0.0]|           1.0|
|[1.0,1.0,83.0]|           1.0|
|[0.0,1.0,57.0]|           0.0|
|     (3,[],[])|           0.0|
+--------------+--------------+
only showing top 20 rows



In [118]:
model_df = output.select("tomorrow_index","today_index","rain_index","features")
model_df.show()

+--------------+-----------+----------+--------------+
|tomorrow_index|today_index|rain_index|      features|
+--------------+-----------+----------+--------------+
|           0.0|        0.0|       4.0| [0.0,0.0,4.0]|
|           0.0|        0.0|       0.0|     (3,[],[])|
|           0.0|        0.0|       0.0|     (3,[],[])|
|           0.0|        0.0|       0.0|     (3,[],[])|
|           0.0|        0.0|       6.0| [0.0,0.0,6.0]|
|           0.0|        0.0|       1.0| [0.0,0.0,1.0]|
|           0.0|        0.0|       0.0|     (3,[],[])|
|           0.0|        0.0|       0.0|     (3,[],[])|
|           1.0|        0.0|       0.0| [1.0,0.0,0.0]|
|           0.0|        1.0|       8.0| [0.0,1.0,8.0]|
|           1.0|        0.0|       0.0| [1.0,0.0,0.0]|
|           1.0|        1.0|      12.0|[1.0,1.0,12.0]|
|           1.0|        1.0|      81.0|[1.0,1.0,81.0]|
|           0.0|        1.0|      20.0|[0.0,1.0,20.0]|
|           2.0|        0.0|       0.0| [2.0,0.0,0.0]|
|         

In [119]:
training_df, test_df = model_df.randomSplit([0.8,0.2],seed = 12345)

In [116]:
training_df.count()

116263

In [117]:
test_df.count()

29197

In [122]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [123]:
from pyspark.mllib.tree import DecisionTree
model = DecisionTree.trainClassifier(training_df, numClasses=2, categoricalFeaturesInfo={},
                                     impurity='gini', maxDepth=5, maxBins=32)

AssertionError: ignored

[DataFrame[Rainfall: string, RainToday: string, RainTomorrow: string, tomorrow_index: double, today_index: double, rain_index: double, features: vector]]

In [113]:
from pyspark.ml import Pipeline

pipeline = Pipeline().setStages(stages)
model = pipeline.fit(training_df)
iw_df = model.transform(test_df)

TypeError: ignored

In [106]:
df_classifier = DecisionTreeClassifier(labelCol="tomorrow_index").fit(training_df)

IllegalArgumentException: ignored

In [58]:
df_predictions = df_classifier.transform(test_df)

In [59]:
# HMMMMM....why did it make everything the same....
df_predictions.show()

+--------+--------------+-----------------+-------------+----------+
|features|tomorrow_index|    rawPrediction|  probability|prediction|
+--------+--------------+-----------------+-------------+----------+
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|
|   [0.0]|           0.0|[88195.0,

In [60]:
df_accuracy = MulticlassClassificationEvaluator(labelCol="tomorrow_index",
                                                metricName="accuracy").evaluate(df_predictions)

In [61]:
df_accuracy

1.0

In [62]:
df_precision = MulticlassClassificationEvaluator(labelCol = "tomorrow_index",
                                                 metricName="weightedPrecision").evaluate(df_predictions)

In [63]:
df_precision

1.0

In [64]:
df_classifier.featureImportances

SparseVector(1, {0: 1.0})

In [65]:
df_auc = MulticlassClassificationEvaluator(labelCol = "tomorrow_index").evaluate(df_predictions)

In [66]:
df_auc

1.0